In [1]:
import pandas as pd

In [ ]:
from pycelonis import get_celonis
# from pycelonis import utils
from pprint import pprint

EMS_API_TOKEN_FILE = "../ems_apikey"
DATA_POOL_ID = "3f9e0c7b-a9d5-4e51-a322-b38e6b10d8b1"
DATA_MODEL_IDS = {
    "accounts_payable": "77da2001-dcf7-4699-b5d3-ce89dc2234d3",
    "accounts_receivable": "524481f7-ed83-4f3b-8452-a6e7897afacb",
    "inventory_management": "a1cd3a04-fbec-4e6e-9fec-65c2fc79ed77",
    "order_management": "213ab550-b1cf-4abc-ad05-23be6a2fd1b8",
    "procurement": "535e9505-0662-432c-b18a-af07edb6137d"
}

celonis = get_celonis(
    base_url="https://pads-x-celonis-group-1.try.celonis.cloud/",
    api_token=open(EMS_API_TOKEN_FILE).read(),
    key_type="APP_KEY"
)

data_pool = celonis.data_integration.get_data_pool(DATA_POOL_ID)

In [ ]:
data_model = data_pool.get_data_model(DATA_MODEL_IDS["accounts_payable"])
# pprint(data_model.get_tables())
# pprint(data_model.get_tables()[0].get_columns())

In [ ]:
from pycelonis.pql.pql import PQL, PQLColumn, PQLFilter

In [ ]:
tables = {t.name: t for t in data_model.get_tables()}

In [ ]:
object_types = [t[10:] for t in tables if t[0] == "o"]
event_types = [t[10:] for t in tables if t[0] == "e"]

In [ ]:
# Create cache for the table columns (table.get_columns() may be very slow)
columns = {}

def get_columns(t):
    if t not in columns:
        columns[t] = tables[t].get_columns()
    return columns[t]

In [ ]:
event_types

In [ ]:
def object_table(ot):
    return f"o_celonis_{ot}"

def event_table(act):
    return f"e_celonis_{act}"

def relation_table():
    return "r_celonis_AP"

def column_query(table, column=None):
    if type(table) == str:
        table = tables[table]
    return PQLColumn(query=f"{table.name}.{column}", name=column)

def all_column_queries(t):
    return [column_query(t, col.name) for col in get_columns(t) if col.name != "ID"]

def execute(query):
    # merge and execute queries. 
    if type(query) == list:
        q = PQL()
        for part in query:
            q += part
        query = q
    elif type(query) == PQLColumn:
        query = PQL() + query
    
#     print(query)
    return data_model.export_data_frame(query)


In [ ]:
# Show head of all event tables

for act in event_types:
    print(f"Activity '{act}'")
    display(execute(all_column_queries(event_table(act))).head())
# head(all_column_queries("o_celonis_VendorAccountCreditItem"))

In [ ]:
# basic PQL test

query = PQL()
# query += PQLColumn(query="o_celonis_VendorAccountCreditItem.*")
query += PQLColumn(query="o_celonis_VendorAccountCreditItem.Type", name="Type")
query += PQLColumn(query="o_celonis_VendorAccountCreditItem.CreationTime", name="CreationTime")
# query += PQLColumn(query="o_celonis_VendorAccountCreditItem.PaymentMethod", name="PaymentMethod")
# query += PQLFilter("FILTER DOMAIN MATCH_PROCESS(NODE['Book Invoice'] \
#                     as src,NODE['Cancel Invoice Receipt']  \
#                     as trg CONNECTED BY EVENTUALLY [ src, trg ]) = 1")
filtered_results = data_model.export_data_frame(query)
filtered_results.head()

In [ ]:
# table_info = pd.DataFrame([{"name": table.name, "columns": [col.name for col in table.get_columns()]} for table in tables])
# table_info["name_short"] = table_info["name"].apply(lambda name: name[:2] + name[10:])

# table_info